In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import librosa
import librosa.display
import IPython.display as ipd
import math
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
import plotly.express as px
from google.colab import drive
from scipy import signal
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import numpy as np
from sklearn.cluster import KMeans
from scipy.signal import windows
from scipy.signal import medfilt, savgol_filter



In [ ]:
drive.mount('/content/drive')

## Arquivos

In [ ]:
arquivos_emg = ["emg__modulo_aberto_1.csv","emg__modulo_aberto_2.csv","emg__modulo_aberto_3.csv",
                "emg__modulo_aberto_4.csv","emg__modulo_aberto_5.csv",
                "emg__modulo_fecha_dedao_resto_aberto_1.csv","emg__modulo_fecha_dedao_resto_aberto_2.csv",
                "emg__modulo_fecha_dedao_resto_aberto_3.csv",
                "emg__modulo_fecha_dedao_resto_aberto_4.csv","emg__modulo_fecha_dedao_resto_aberto_5.csv"]

In [ ]:
#arquivos_emg = ["emg__modulo_fecha_dedao_resto_aberto_1.csv"]

In [ ]:
arquivos_abre_fecha_emg = arquivos_emg[:]

In [ ]:
FRAME_SIZE = 512*2
HOP_LENGTH = int(0.25*FRAME_SIZE)
hann_wnd = windows.hann(FRAME_SIZE,sym=False)
## o tanto que salta para a direita de um frame para o outro, frames serão sobrepostos
## diferentes frames terão amostras iguais

In [ ]:
path = "drive/My Drive/TCC - Vinícius Franklin - Desenvolvimento de Prótese Ativa Controlada por Sinais EMG Utilizando Classificação com Machine Learning/Testes Dados - EMG/conf_1_dedao/"

In [ ]:
dfs_emg_module  = {}
for arquivo in arquivos_emg:


  df = pd.read_csv(path + arquivo)
  # Garante que os dados estão no formato numérico
  df["tempo_arduino_micro_s"] = pd.to_numeric(df["tempo_arduino_micro_s"], errors="coerce")
  df["valor_ADC"] = pd.to_numeric(df["valor_ADC"], errors="coerce")


  df["tempo_arduino_s"] = df["tempo_arduino_micro_s"]/10**6

  periodo = float(df.loc[106,"tempo_arduino_s"] - df.loc[105,"tempo_arduino_s"])
  sr = 1/periodo

  tempo = df["tempo_arduino_s"].values
  fs_eff = 1 / np.mean(np.diff(tempo))
  sr = fs_eff

  dfs_emg_module[arquivo] = {"df":df,"periodo":periodo,"sr":sr}
  dfs_emg_module[arquivo]['features'] = {}




In [ ]:
print(f"Sample Rate: {sr} hz")

In [ ]:
frame_size_time = FRAME_SIZE*1/sr
print(f"Frame Size no tempo: {frame_size_time} s")
hop_lenght_time = HOP_LENGTH*1/sr
print(f"HOP LENGHT no tempo: {hop_lenght_time} s")

In [ ]:
def plot_magnitude_spectrum(signal, sr, title, f_ratio=1,norm=0):
    # FFT
    X = np.fft.fft(signal)
    X_mag = np.abs(X)


    # Frequência
    # Normalizando pelo Valor Total
    f = np.linspace(0, sr, len(X_mag))
    f_bins = int(len(X_mag) * f_ratio)

    X_mag = np.abs(X_mag[:f_bins//2])**2

    if norm != 0:
      X_mag = X_mag/sum(X_mag)

    # Plotly
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=f[:f_bins//2],
        y=X_mag,
        mode='lines',
        name='Magnitude Normalizada'
    ))

    fig.update_layout(
        title=title,
        xaxis_title='Frequency (Hz)',
        yaxis_title='Magnitude Normalizada',
        width=900,
        height=400
    )

    fig.show()
    # retornar so a parte positiva, ja que é simetrico
    return X_mag[:f_bins//2],f,f_bins


In [ ]:
def simple_moving_average(data, window_size=6):
    weights = np.ones(window_size) / window_size
    sma = np.convolve(data, weights, mode='same')
    return sma

def median_clean(signal, kernel_size=7):
  # kernel_size deve ser ímpar
  k = kernel_size if kernel_size % 2 == 1 else kernel_size + 1
  return medfilt(signal, kernel_size=k)


def majority_vote(labels, window=5):
    half = window // 2
    padded = np.pad(labels, (half, half), mode='edge')
    out = np.zeros_like(labels)
    for i in range(len(labels)):
        win = padded[i:i+window]
        out[i] = np.bincount(win).argmax()
    return out

def run_length_smooth(labels, min_run_length=5):
    # labels: 1D array of 0/1
    out = labels.copy()
    n = len(labels)
    i = 0
    while i < n:
        j = i
        while j < n and labels[j] == labels[i]:
            j += 1
        run_len = j - i
        if run_len < min_run_length:
            out[i:j] = 1 - labels[i]  # flip short run
        i = j
    return out



## Original

In [ ]:

for arquivo in arquivos_emg[:]:
    df = dfs_emg_module[arquivo]["df"]

    # Vetores já existentes
    t = df["tempo_arduino_s"].values
    y = df["valor_ADC"].values

    # Remove a componente média (DC) apenas no vetor
    y_sem_dc = y - y.mean()

    # Cria o gráfico interativo usando arrays diretamente
    fig = px.line(
        x=t,
        y=y_sem_dc,
        title=f"Sinal EMG (sem componente média) — {arquivo}",
        labels={"x": "Tempo (s)", "y": "Amplitude (uADC)"}
    )
    fig.update_layout(template="plotly_white")
    fig.show()


In [ ]:
for data in arquivos_emg[:]:
  valor = dfs_emg_module[data]["df"].valor_ADC.values
  valor = valor - valor.mean()
  plot_magnitude_spectrum(valor ,dfs_emg_module[data]["sr"],data,norm=1)

## Filtro

Resumo da decisão sobre filtragem EMG no pipeline

Para o processamento dos sinais EMG, inicialmente utilizei um filtro Butterworth passa-faixa (20–500 Hz, ordem 8) com sosfiltfilt, garantindo fase zero e evitando distorções temporais. Porém, esse tipo de filtragem não pode ser embarcado no microcontrolador, já que exige filtrar o sinal para frente e para trás.

Por isso, a solução adotada para uso real é aplicar o mesmo filtro Butterworth, mas agora de forma causal com sosfilt, que é compatível com implementação embarcada (IIR em cascata de SOS). O filtro causal possui atraso de grupo e distorção de fase, mas, como o modelo de machine learning é treinado com esse mesmo filtro, ele aprende esses padrões e não há perda prática de desempenho.

Para validar a robustez do pipeline, posso comparar os clusters obtidos com o dataset filtrado por sosfilt (causal) e com sosfiltfilt (fase zero). Se os agrupamentos forem semelhantes, isso confirma que a distorção de fase do filtro causal não compromete a separação das classes.

Assim, para uso embarcado e treinamento consistente, o processamento adotado é:

Filtro IIR Butterworth passa-faixa (20–500 Hz)

Implementação causal utilizando signal.sosfilt

Frames de 1024 samples com hop de 25%

Essa abordagem mantém compatibilidade com o hardware e garante uniformidade entre treino e execução em tempo real.

In [ ]:
from scipy import signal

for data in arquivos_emg[:]:
  valor = dfs_emg_module[data]["df"].valor_ADC.values
  valor = valor - valor.mean()
  # Butterworth bandstop como Second-Order Sections
  sos = signal.butter(N=8, Wn=[20, 500], btype='bandpass', fs=dfs_emg_module[data]["sr"], output='sos')



  # Aplica o filtro
  filtered_nc = signal.sosfiltfilt(sos = sos, x = valor)
  filtered_c = signal.sosfilt(sos = sos, x = valor)
  # Média Móvel
  #filtered = simple_moving_average(data=filtered)

  #sos = signal.butter(N=8, Wn=[70, 90], btype='bandstop', fs=sr, output='sos')
  #filtered_nc = signal.sosfiltfilt(sos = sos, x = filtered_nc)
  #filtered_c = signal.sosfilt(sos = sos, x = filtered_c)
  ## Filtro Mediano
  ##filtered = median_clean(filtered)
#
  #sos = signal.butter(N=1, Wn=[225, 245], btype='bandstop', fs=sr, output='sos')
  #filtered_nc = signal.sosfiltfilt(sos = sos, x = filtered_nc)
  #filtered_c = signal.sosfilt(sos = sos, x = filtered_c)
#
  #sos = signal.butter(N=8, Wn=[140, 160], btype='bandstop', fs=sr, output='sos')
  #filtered_nc = signal.sosfiltfilt(sos = sos, x = filtered_nc)
  #filtered_c = signal.sosfilt(sos = sos, x = filtered_c)
#
#
  #sos = signal.butter(N=1, Wn=[305, 320], btype='bandstop', fs=sr, output='sos')
  #filtered_nc = signal.sosfiltfilt(sos = sos, x = filtered_nc)
  #filtered_c = signal.sosfilt(sos = sos, x = filtered_c)

  dfs_emg_module[data]["filtered"] = filtered_c
  dfs_emg_module[data]["filtered_Nao_Causal"] = filtered_nc
  plot_magnitude_spectrum(dfs_emg_module[data]["filtered"] ,dfs_emg_module[data]["sr"],data,norm=1)

In [ ]:
for data in arquivos_abre_fecha_emg[:]:

    sinal = dfs_emg_module[data]["filtered_Nao_Causal"]

    fig = go.Figure()

    # Plot do sinal EMG
    fig.add_trace(go.Scatter(
        x=np.arange(len(sinal))/sr,
        y=sinal,
        mode='lines',
        name='EMG Não Causal',
        line=dict(color='blue', width=1)
    ))

    fig.update_layout(
        title=f"{data} — Sinal EMG Depois do Filtro Não Causal",
        xaxis_title="Tempo (s)",
        yaxis_title="Amplitude (uADC)",
        height=500
    )

    # Mostra o gráfico no navegador
    fig.show()


    sinal = dfs_emg_module[data]["filtered"]

    fig = go.Figure()

    # Plot do sinal EMG
    fig.add_trace(go.Scatter(
        x=np.arange(len(sinal))/sr,
        y=sinal,
        mode='lines',
        name='EMG Causal',
        line=dict(color='blue', width=1)
    ))

    fig.update_layout(
        title=f"{data} — Sinal EMG Depois do Filtro Causal",
        xaxis_title="Tempo (s)",
        yaxis_title="Amplitude (uADC)",
        height=500
    )




    # Mostra o gráfico no navegador
    fig.show()

## Amplitude Envelope

In [ ]:
## calculate the amplitude envelope, but the module, so because its centered in 0
def amplitude_envelope(signal, frame_size, hop_length):
    amplitude_envelope = []

    # calculate the AE for each frame

    for i in range(0, len(signal) - frame_size + 1, hop_length):
        wnd = signal[i:i+frame_size]
        wnd = wnd*hann_wnd
        max_value = max(wnd)
        min_value = min(wnd)
        current_frame_amplitude_envelope = max_value if max_value > np.abs(min_value) else min_value
        amplitude_envelope.append(current_frame_amplitude_envelope)

    return np.array(amplitude_envelope)


def fancy_amplitude_envelope(signal, frame_size, hop_length):
    return np.array([max(signal[i:i+frame_size]) for i in range(0,signal.size, hop_length)])

In [ ]:
for arquivo in arquivos_emg:
  filt_values = dfs_emg_module[arquivo]["filtered"]
  amp_en = amplitude_envelope(filt_values,FRAME_SIZE,HOP_LENGTH)
  dfs_emg_module[arquivo]['features']['amp_en']  = amp_en


In [ ]:
amplitude_envelope(df["valor_ADC"].values,FRAME_SIZE,HOP_LENGTH)

In [ ]:
for arquivo in arquivos_emg:
  ae = dfs_emg_module[arquivo]['features']["amp_en"]
  adc_valus = dfs_emg_module[arquivo]["filtered"]
  sr = dfs_emg_module[arquivo]["sr"]
  t = np.arange(len(ae)) * HOP_LENGTH / sr
  # Cria subplots
  fig = go.Figure()

  # ECG filtrado
  fig.add_trace(go.Scatter(x=np.arange(len(adc_valus))/sr, y=adc_valus,
                          mode='lines', name='EMG', line=dict(color='blue'), opacity=0.5))

  # Envelope sobreposto
  fig.add_trace(go.Scatter(x=t, y=ae,
                          mode='lines', name='AE', line=dict(color='red', width=2)))

  fig.update_layout(title=arquivo)

  fig.show()

## RMS

In [ ]:
### rms energy, my code

def rms_calculator(data):
    soma = 0
    for sample in data:
        soma += sample**2
    return (soma/len(data))**0.5

def frame_2_rms(data,frame_size,hop_lenght):
    lista_rms = []
    for i in range(0,len(data) - frame_size + 1,hop_lenght):
        frame = data[i:i + frame_size]
        frame = frame*hann_wnd
        rms_value = rms_calculator(data=frame)
        lista_rms.append(rms_value)
    return np.array(lista_rms)


In [ ]:

for arquivo in arquivos_emg:
  filt_values = dfs_emg_module[arquivo]["filtered"]
  rms = frame_2_rms(data=filt_values, frame_size=FRAME_SIZE, hop_lenght=HOP_LENGTH)
  dfs_emg_module[arquivo]['features']['rms'] = rms

In [ ]:
for arquivo in arquivos_emg:
  rms = dfs_emg_module[arquivo]['features']["rms"]
  adc_valus = dfs_emg_module[arquivo]["filtered"]
  sr = dfs_emg_module[arquivo]["sr"]
  t = np.arange(len(rms)) * HOP_LENGTH / sr
  # Cria subplots
  fig = go.Figure()

  # ECG filtrado
  fig.add_trace(go.Scatter(x=np.arange(len(adc_valus))/sr, y=adc_valus,
                          mode='lines', name='EMG', line=dict(color='blue'), opacity=0.5))

  # Envelope sobreposto
  fig.add_trace(go.Scatter(x=t, y=rms,
                          mode='lines', name='RMS', line=dict(color='red', width=2)))

  fig.update_layout(title=arquivo)

  fig.show()

## Zero Crossing Rate

In [ ]:
### implementing zero-crossing rate


def zero_cs_sum(frame):
    count = 0
    for n in range(len(frame)-1):
        count += 0.5 * (abs(np.sign(frame[n]) - np.sign(frame[n+1])))
    return count / (len(frame)-1)  # agora é taxa (0..1)


def zcr_calculator(data,frame_size,hop_lenght):
    lista_zc = []
    for i in range(0,len(data) - frame_size + 1,hop_lenght):
        frame = data[i:i + frame_size]
        frame = frame*hann_wnd
        zc_value = zero_cs_sum(frame=frame)
        lista_zc.append(zc_value)
    return np.array(lista_zc)


In [ ]:
for arquivo in arquivos_emg:
  filt_values = dfs_emg_module[arquivo]["filtered"]
  zcr = zcr_calculator(data=filt_values, frame_size=FRAME_SIZE, hop_lenght=HOP_LENGTH,)
  dfs_emg_module[arquivo]['features']["zcr"] = zcr

In [ ]:
for arquivo in arquivos_emg:
  zcr = dfs_emg_module[arquivo]['features']["zcr"]
  adc_valus = dfs_emg_module[arquivo]["filtered"]
  sr = dfs_emg_module[arquivo]["sr"]
  t = np.arange(len(zcr)) * HOP_LENGTH / sr
  # Cria subplots
  fig = go.Figure()

  # ECG filtrado
  #fig.add_trace(go.Scatter(x=np.arange(len(adc_valus))/sr, y=adc_valus,
     #                     mode='lines', name='EMG', line=dict(color='blue'), opacity=0.5))

  # Envelope sobreposto
  fig.add_trace(go.Scatter(x=t, y=zcr,
                          mode='lines', name='ZCR', line=dict(color='red', width=2)))

  fig.update_layout(title=arquivo)

  fig.show()

## Spectograma

In [ ]:
def plot_spectrogram(Y, sr, hop_length, arquivo,y_axis="linear", vmin=None, vmax=None):
    plt.figure(figsize=(25, 10))
    librosa.display.specshow(Y,
                             sr=sr,
                             hop_length=hop_length,
                             x_axis="time",
                             y_axis=y_axis,
                             vmin=vmin,
                             vmax=vmax,cmap="inferno")  # <- controla o range de cores
    plt.title(arquivo)
    plt.colorbar(format="%+2.f")


In [ ]:
for arquivo in arquivos_emg:
  filt_values = dfs_emg_module[arquivo]["filtered"]
  S_emg = librosa.stft(y=filt_values, n_fft=FRAME_SIZE, hop_length=HOP_LENGTH,window='hann')
  Y_emg = librosa.power_to_db(np.abs(S_emg) ** 2)

  dfs_emg_module[arquivo]['Spectograma'] = {"S_emg":S_emg,"Y_emg":Y_emg}

  vmin = Y_emg.min()
  vmax = Y_emg.max()

  plot_spectrogram(Y_emg, sr, HOP_LENGTH, y_axis="log", vmin=vmin, vmax=vmax, arquivo=arquivo)

## Band Energy Ratio

In [ ]:
def calculate_split_frequency_bin(split_frequency, sample_rate, num_frequency_bins):
    """Infer the frequency bin associated to a given split frequency."""

    frequency_range = sample_rate / 2
    frequency_delta_per_bin = frequency_range / num_frequency_bins
    split_frequency_bin = math.floor(split_frequency / frequency_delta_per_bin)
    return int(split_frequency_bin)


def band_energy_ratio(spectrogram, split_frequency, sample_rate):
    """Calculate band energy ratio with a given split frequency."""

    split_frequency_bin = calculate_split_frequency_bin(split_frequency, sample_rate,spectrogram.shape[0])
    band_energy_ratio = []

    # calculate power spectrogram
    power_spectrogram = np.abs(spectrogram) ** 2
    power_spectrogram = power_spectrogram.T

    # calculate BER value for each frame
    for frame in power_spectrogram:
        sum_power_low_frequencies = frame[:split_frequency_bin].sum()
        sum_power_high_frequencies = frame[split_frequency_bin:].sum()
        #print(sum_power_low_frequencies,sum_power_high_frequencies)
        epsilon = 1e-12
        band_energy_ratio_current_frame = sum_power_low_frequencies / (sum_power_high_frequencies + epsilon)

        band_energy_ratio.append(band_energy_ratio_current_frame)


    return np.array(band_energy_ratio)

In [ ]:
split_frequency = 70


for arquivo in arquivos_emg:
  S_ecg = dfs_emg_module[arquivo]["Spectograma"]["S_emg"]
  sr = dfs_emg_module[arquivo]["sr"]
  ber_emg = band_energy_ratio(S_ecg, split_frequency, sr)
  dfs_emg_module[arquivo]['features']["Band Energy Ratio"] = ber_emg


In [ ]:
for arquivo in arquivos_emg:
  ber = dfs_emg_module[arquivo]['features']["Band Energy Ratio"]
  sr = dfs_emg_module[arquivo]["sr"]
  t = np.arange(len(ber)) * HOP_LENGTH / sr
  # Cria subplots
  fig = go.Figure()


  # Envelope sobreposto
  fig.add_trace(go.Scatter(x=t, y=ber,
                          mode='lines', name='Bend Energy Ratio', line=dict(color='red', width=2)))

  fig.update_layout(title=arquivo)

  fig.show()

# Spectral Centroid

In [ ]:

for arquivo in arquivos_emg:
  filt_values = dfs_emg_module[arquivo]["filtered"]
  sr = dfs_emg_module[arquivo]["sr"]
  sc = librosa.feature.spectral_centroid(y=filt_values, sr=sr, n_fft=FRAME_SIZE, hop_length=HOP_LENGTH,window="hann")[0]

  dfs_emg_module[arquivo]['features']["Spectral Centroid"] = sc


In [ ]:
for arquivo in arquivos_emg:
  sc = dfs_emg_module[arquivo]['features']["Spectral Centroid"]
  sr = dfs_emg_module[arquivo]["sr"]
  t = np.arange(len(sc)) * HOP_LENGTH / sr
  # Cria subplots
  fig = go.Figure()


  # Envelope sobreposto
  fig.add_trace(go.Scatter(x=t, y=sc,
                          mode='lines', name='Spectral Centroid', line=dict(color='red', width=2)))

  fig.update_layout(title=arquivo)

  fig.show()

# Bandwidth

In [ ]:

for arquivo in arquivos_emg:
  filt_values = dfs_emg_module[arquivo]["filtered"]
  sr = dfs_emg_module[arquivo]["sr"]
  bw = librosa.feature.spectral_bandwidth(y=filt_values, sr=sr, n_fft=FRAME_SIZE, hop_length=HOP_LENGTH,window='hann')[0]
  dfs_emg_module[arquivo]['features']["Bandwidth"] = bw


In [ ]:
for arquivo in arquivos_emg:
  bw = dfs_emg_module[arquivo]['features']["Bandwidth"]
  sr = dfs_emg_module[arquivo]["sr"]
  t = np.arange(len(bw)) * HOP_LENGTH / sr
  # Cria subplots
  fig = go.Figure()


  # Envelope sobreposto
  fig.add_trace(go.Scatter(x=t, y=bw,
                          mode='lines', name='Spectral Centroid', line=dict(color='red', width=2)))

  fig.update_layout(title=arquivo)

  fig.show()

# f0

In [ ]:
import numpy as np

def peak_frequency(signal, sr, frame_size, hop_length):
    """
    Calcula a frequência de pico e magnitude para cada frame do sinal.

    Parâmetros:
    -----------
    signal : np.array
        Sinal 1D
    sr : int
        Taxa de amostragem
    frame_size : int
        Tamanho de cada frame (em samples)
    hop_length : int
        Salto entre frames (em samples)

    Retorna:
    --------
    peak_freqs : np.array
        Frequência de pico de cada frame
    peak_mags : np.array
        Magnitude correspondente de cada frame
    """
    n_samples = len(signal)
    frames = []

    # Cria frames com overlap
    for start in range(0, n_samples - frame_size + 1, hop_length):
        frames.append(signal[start:start+frame_size])
    frames = np.array(frames)

    # Arrays de saída
    peak_freqs = np.zeros(len(frames))
    peak_mags = np.zeros(len(frames))

    # Calcula FFT para cada frame
    for i, frame in enumerate(frames):
        frame = frame*hann_wnd
        X = np.fft.fft(frame)
        X_mag = np.abs(X)[:frame_size // 2]  # metade positiva
        sum_mag = sum(X_mag)
        freqs = np.fft.fftfreq(frame_size, d=1/sr)[:frame_size // 2]
        idx = np.argmax(X_mag)
        peak_freqs[i] = freqs[idx]
        peak_mags[i] = X_mag[idx]
        peak_mags_norm = (peak_mags/sum_mag)*100
    return peak_freqs, peak_mags_norm


In [ ]:
for arquivo in arquivos_emg:
  filt_values = dfs_emg_module[arquivo]["filtered"]
  sr = dfs_emg_module[arquivo]["sr"]
  f0,X_f0 = peak_frequency(signal=filt_values, sr=sr, frame_size=FRAME_SIZE, hop_length=HOP_LENGTH)
  dfs_emg_module[arquivo]['features']["f0"] = f0
  dfs_emg_module[arquivo]['features']["Mag_norm_f0"] = X_f0

In [ ]:
for arquivo in arquivos_emg:
  f0 = dfs_emg_module[arquivo]['features']["f0"]
  X_mag_f0 = dfs_emg_module[arquivo]['features']["Mag_norm_f0"]
  sr = dfs_emg_module[arquivo]["sr"]
  t = np.arange(len(f0)) * HOP_LENGTH / sr
  # Cria subplots
  fig = go.Figure()


  # Envelope sobreposto
  fig.add_trace(go.Scatter(x=t, y=f0,
                          mode='lines', name='f0', line=dict(color='red', width=2)))

  fig.add_trace(go.Scatter(x=t, y=X_mag_f0,
                          mode='lines', name='Mag Norm f0', line=dict(color='red', width=2)))

  fig.update_layout(title=arquivo)

  fig.show()

In [ ]:
arquivos_emg

In [ ]:
#for data in arquivos_emg[:]:
#  valor = dfs_emg_module[data]["df"].valor_ADC.values
#  valor = valor - valor.mean()
#  # Butterworth bandstop como Second-Order Sections
#  sos = signal.butter(N=8, Wn=[20, 500], btype='bandpass', fs=dfs_emg_module[data]["sr"], output='sos')
#
#  # Aplica o filtro
#  filtered = signal.sosfiltfilt(sos = sos, x = valor)
#
#  # Butterworth bandstop como Second-Order Sections
#
#  #sos = signal.butter(N=8, Wn=[160, 168], btype='bandstop', fs=dfs_emg_module[data]["sr"], output='sos')
#
#  # Aplica o filtro
#  #filtered_2 = signal.sosfiltfilt(sos = sos, x = filtered)
#
#
#  dfs_emg_module[data]["filtered"] = filtered
#  plot_magnitude_spectrum(dfs_emg_module[data]["filtered"][10*FRAME_SIZE:12*FRAME_SIZE] ,dfs_emg_module[data]["sr"],data,norm=1)

# Potencia Hamônicos

In [ ]:
def band_power(f1, f2, freqs, total_power, P):
    idx = np.logical_and(freqs >= f1, freqs <= f2)
    return np.sum(P[idx]) / total_power if total_power != 0 else 0

def features_harmonicos(signal, sr, frame_size, hop_length):
    n_samples = len(signal)
    frames = []

    # Cria frames com overlap
    for start in range(0, n_samples - frame_size + 1, hop_length):
        frames.append(signal[start:start+frame_size])
    frames = np.array(frames)

    arrays = {
        "bp_50_150": np.zeros(len(frames)),
        "bp_150_200": np.zeros(len(frames)),
        "bp_200_300": np.zeros(len(frames)),
        "bp_300_500": np.zeros(len(frames)),
        "bp_novo": np.zeros(len(frames)),
        "bp_1_harm": np.zeros(len(frames)),
        "bp_2_harm": np.zeros(len(frames)),
        "bp_3_harm": np.zeros(len(frames)),
        "bp_4_harm": np.zeros(len(frames)),
        "harm_energy": np.zeros(len(frames)),
        "noise_energy": np.zeros(len(frames)),
        "hnr": np.zeros(len(frames)),
        "mdf": np.zeros(len(frames)),
        "mnf": np.zeros(len(frames)),
    }

    for i, frame in enumerate(frames):
        frame = frame*hann_wnd
        X = np.fft.fft(frame)
        X_mag = np.abs(X)[:frame_size // 2]
        P = X_mag**2
        total_power = np.sum(P)
        freqs = np.fft.fftfreq(frame_size, d=1/sr)[:frame_size // 2]

        bp_50_150 = band_power(0, 50, freqs, total_power, P)
        bp_150_200 = band_power(70, 110, freqs, total_power, P)
        bp_200_300 = band_power(130, 170, freqs, total_power, P)
        bp_300_500 = band_power(190, 230, freqs, total_power, P)
        bp_novo = band_power(250, 500, freqs, total_power, P)

        bp_1_harm = band_power(50, 70, freqs, total_power, P)
        bp_2_harm = band_power(110, 130, freqs, total_power, P)
        bp_3_harm = band_power(170, 190, freqs, total_power, P)
        bp_4_harm = band_power(230, 250, freqs, total_power, P)



        harm_energy = bp_1_harm + bp_2_harm + bp_3_harm + bp_4_harm
        noise_energy = bp_50_150 + bp_150_200 + bp_200_300 + bp_300_500 + bp_novo
        hnr = (harm_energy / (noise_energy + 1e-12)) * 100

        cumsum = np.cumsum(P)
        mdf = freqs[np.searchsorted(cumsum, cumsum[-1] / 2)]
        mnf = np.sum(freqs * P) / (np.sum(P) + 1e-12)

        arrays["bp_50_150"][i] = bp_50_150
        arrays["bp_150_200"][i] = bp_150_200
        arrays["bp_200_300"][i] = bp_200_300
        arrays["bp_300_500"][i] = bp_300_500
        arrays["bp_novo"][i] = bp_novo
        arrays["bp_1_harm"][i] = bp_1_harm
        arrays["bp_2_harm"][i] = bp_2_harm
        arrays["bp_3_harm"][i] = bp_3_harm
        arrays["bp_4_harm"][i] = bp_4_harm
        arrays["harm_energy"][i] = harm_energy
        arrays["noise_energy"][i] = noise_energy
        arrays["hnr"][i] = hnr
        arrays["mdf"][i] = mdf
        arrays["mnf"][i] = mnf

    return arrays


In [ ]:
for arquivo in arquivos_emg:
  filt_values = dfs_emg_module[arquivo]["filtered"]
  sr = dfs_emg_module[arquivo]["sr"]
  arrays = features_harmonicos(signal=filt_values, sr=sr, frame_size=FRAME_SIZE, hop_length=HOP_LENGTH)
  dfs_emg_module[arquivo]["features"] = dfs_emg_module[arquivo]["features"] | arrays

In [ ]:
# Lista das features que você quer plotar
freq_features = [
    "bp_50_150", "bp_150_200", "bp_200_300", "bp_300_500","bp_novo",
    "bp_1_harm", "bp_2_harm", "bp_3_harm","bp_4_harm",
    "harm_energy", "noise_energy", "hnr", "mdf", "mnf"
]

for arquivo in arquivos_emg:
    sr = dfs_emg_module[arquivo]["sr"]
    features = dfs_emg_module[arquivo]["features"]

    # Número de frames (assume que todas as features têm o mesmo tamanho)
    n_frames = len(features[freq_features[0]])
    t = np.arange(n_frames) * HOP_LENGTH / sr

    # Cria figura
    fig = go.Figure()

    # Adiciona cada feature de frequência
    colors = ['blue','green','orange','purple','brown','pink','gray','cyan','magenta','lime','teal','olive']
    for i, feature_name in enumerate(freq_features):
        fig.add_trace(go.Scatter(
            x=t,
            y=features[feature_name],
            mode='lines',
            name=feature_name,
            line=dict(color=colors[i % len(colors)], width=2)
        ))

    # Adiciona envelope de f0 se existir
    if "f0" in dfs_emg_module[arquivo]:
        f0 = dfs_emg_module[arquivo]["f0"]
        X_mag_f0 = dfs_emg_module[arquivo]["X_mag_f0"]
        fig.add_trace(go.Scatter(
            x=t,
            y=f0,
            mode='lines',
            name='f0',
            line=dict(color='red', width=2)
        ))
        fig.add_trace(go.Scatter(
            x=t,
            y=X_mag_f0,
            mode='lines',
            name='Mag Norm f0',
            line=dict(color='red', width=2, dash='dot')
        ))

    fig.update_layout(
        title=arquivo,
        xaxis_title='Tempo [s]',
        yaxis_title='Valor da Feature'
    )

    fig.show()


# Mais Features no Tempo

In [ ]:
import numpy as np

def outras_features_tempo(signal, sr, frame_size, hop_length):
    n_samples = len(signal)
    frames = []

    # Cria frames com overlap
    for start in range(0, n_samples - frame_size + 1, hop_length):
        frames.append(signal[start:start+frame_size])
    frames = np.array(frames)

    # Inicializa arrays de saída
    arrays = {
        "mav": np.zeros(len(frames)),
        "wl": np.zeros(len(frames)),
        "mean_v": np.zeros(len(frames)),
        "std_v": np.zeros(len(frames)),
        "var_v": np.zeros(len(frames)),
        "ssc": np.zeros(len(frames)),
        "mavs": np.zeros(len(frames))  # MAV ponderada
    }

    # Calcula features para cada frame
    for i, frame in enumerate(frames):
        frame = frame*hann_wnd
        mav = np.mean(np.abs(frame))                         # Mean Absolute Value
        wl = np.sum(np.abs(np.diff(frame)))                  # Waveform Length
        mean_v = np.mean(frame)                              # Média
        std_v = np.std(frame)                                # Desvio padrão
        var_v = np.var(frame)                                # Variância
        ssc = np.sum(np.diff(np.sign(np.diff(frame))) != 0)  # Slope Sign Changes

        # MAV ponderada (MAVS) - exemplo simples: pondera central do frame
        weights = np.linspace(0.5, 1.5, len(frame))
        mavs = np.mean(np.abs(frame) * weights)

        arrays["mav"][i] = mav
        arrays["wl"][i] = wl
        arrays["mean_v"][i] = mean_v
        arrays["std_v"][i] = std_v
        arrays["var_v"][i] = var_v
        arrays["ssc"][i] = ssc
        arrays["mavs"][i] = mavs

    return arrays


In [ ]:
for arquivo in arquivos_emg:
  filt_values = dfs_emg_module[arquivo]["filtered"]
  sr = dfs_emg_module[arquivo]["sr"]
  arrays = outras_features_tempo(signal=filt_values, sr=sr, frame_size=FRAME_SIZE, hop_length=HOP_LENGTH)
  dfs_emg_module[arquivo]["features"] = dfs_emg_module[arquivo]["features"] | arrays

In [ ]:
# Lista das features de tempo que queremos plotar
tempo_features = ["mav", "wl", "mean_v", "std_v", "var_v", "ssc", "mavs"]

for arquivo in arquivos_emg:
    sr = dfs_emg_module[arquivo]["sr"]
    features = dfs_emg_module[arquivo]["features"]

    # Número de frames (assume que todas as features têm o mesmo tamanho)
    n_frames = len(features[tempo_features[0]])
    t = np.arange(n_frames) * HOP_LENGTH / sr

    # Cria figura
    fig = go.Figure()

    # Adiciona cada feature do tempo
    colors = ['blue','green','orange','purple','brown','pink','gray','cyan']
    for i, feature_name in enumerate(tempo_features):
        fig.add_trace(go.Scatter(
            x=t,
            y=features[feature_name],
            mode='lines',
            name=feature_name,
            line=dict(color=colors[i % len(colors)], width=2)
        ))

    fig.update_layout(
        title=f"{arquivo} - Features do Tempo",
        xaxis_title='Tempo [s]',
        yaxis_title='Valor da Feature'
    )

    fig.show()


# Features

In [ ]:
dfs_emg_module[arquivo]["features"].keys()

In [ ]:
time_features = ['amp_en', 'rms', 'zcr','mav', 'wl', 'mean_v', 'std_v', 'var_v', 'ssc', 'mavs']
freq_features = ['Band Energy Ratio', 'Spectral Centroid', 'Bandwidth', 'f0', 'Mag_norm_f0', 'bp_50_150', 'bp_150_200', 'bp_200_300',
	'bp_300_500', 'bp_1_harm', 'bp_2_harm', 'bp_3_harm', 'harm_energy', 'noise_energy', 'hnr', 'mdf', 'mnf']

In [ ]:
from sklearn.preprocessing import MinMaxScaler


## Dicionário de features relevantes por arquivo
#features_relevantes = {
#    "emg__modulo_1_abre_fecha.csv": [
#        "amp_en", "rms", "Mag_norm_f0", "mdf", "mnf", "bp_50_150",
#        "mav", "wl", "mean_v", "std_v", "var_v", "mavs", "Spectral Centroid"
#    ],
#    "emg__modulo_1_abre_fecha_2.csv": [
#        "amp_en", "rms", "Mag_norm_f0", "harm_energy", "noise_energy", "hnr", "mdf",
#        "mav", "wl", "mean_v", "std_v", "var_v", "ssc","mavs"
#    ],
#    "emg__modulo_1_abre_fecha_variando.csv": [
#        "amp_en", "rms", "Mag_norm_f0", "harm_energy", "noise_energy", "hnr", "mdf",
#        "mav", "wl", "mean_v", "std_v", "var_v", "ssc", "mavs"
#    ]
#}

for arquivo in arquivos_emg:
    # Pega as features do arquivo atual
    features = list(dfs_emg_module[arquivo]["features"].keys())

    # Filtra e Ajusta as features
    dfs_emg_module[arquivo]["features_filtered"] = {}
    for f in features:
      value = dfs_emg_module[arquivo]["features"][f][:]


      value = simple_moving_average(data = value,window_size=1)
      value = median_clean(signal = value, kernel_size=3)

      #value = value ** 2

      dfs_emg_module[arquivo]["features_filtered"][f] = value

    # Encontra o tamanho mínimo entre as features
    len_min = min(len(dfs_emg_module[arquivo]["features_filtered"][f]) for f in features)

    # Trunca todas as features para o mesmo tamanho
    features_truncadas = [dfs_emg_module[arquivo]["features_filtered"][f][:len_min] for f in features]

    # Junta em uma matriz (frames x features)
    dfs_emg_module[arquivo]["X"] = np.column_stack(features_truncadas)



    # Encontra o tamanho mínimo entre as features
    len_min = min(len(dfs_emg_module[arquivo]["features"][f]) for f in features)

    # Trunca todas as features para o mesmo tamanho
    features_brutas_truncadas = [dfs_emg_module[arquivo]["features"][f][:len_min] for f in features]

    # Junta em uma matriz (frames x features)
    dfs_emg_module[arquivo]["X_Brutas"] = np.column_stack(features_brutas_truncadas)


    # Normaliza
    scaler = MinMaxScaler()
    dfs_emg_module[arquivo]["X_scaled"] = scaler.fit_transform(dfs_emg_module[arquivo]["X"])
    print(f"{arquivo}: matriz de features -> {dfs_emg_module[arquivo]['X_scaled'].shape}")


In [ ]:
## Todos os Arquivos

for arquivo in arquivos_emg:
    sr = dfs_emg_module[arquivo]["sr"]
    #features = features_relevantes[arquivo]
    features = list(dfs_emg_module[arquivo]["features_filtered"].keys())
    # Obtém matriz normalizada e tamanho
    X_scaled = dfs_emg_module[arquivo]["X_scaled"]
    n_frames = X_scaled.shape[0]
    t = np.arange(n_frames) * HOP_LENGTH / sr

    # Cria figura
    fig = go.Figure()

    # Paleta de cores
    colors = ['blue', 'green', 'orange', 'purple', 'brown', 'pink', 'gray',
              'cyan', 'magenta', 'lime', 'teal', 'olive', 'navy', 'gold', 'maroon']

    # Adiciona cada feature normalizada
    for i, feature_name in enumerate(features):
        fig.add_trace(go.Scatter(
            x=t,
            y=X_scaled[:, i],
            mode='lines',
            name=feature_name,
            line=dict(color=colors[i % len(colors)], width=2)
        ))

    fig.update_layout(
        title=f"{arquivo} — Features Normalizadas",
        xaxis_title="Tempo [s]",
        yaxis_title="Valor Normalizado (0–1)",
        height=600
        #legend=dict(orientation='v', yanchor='bottom', y=-0.3, xanchor='center', x=0.5)
    )

    fig.show()


In [ ]:
## APENAS O ABRE FECHA

for arquivo in arquivos_abre_fecha_emg:
    sr = dfs_emg_module[arquivo]["sr"]
    #features = features_relevantes[arquivo]
    features = list(dfs_emg_module[arquivo]["features_filtered"].keys())
    # Obtém matriz normalizada e tamanho
    X_scaled = dfs_emg_module[arquivo]["X_scaled"]
    n_frames = X_scaled.shape[0]
    t = np.arange(n_frames) * HOP_LENGTH / sr

    # Cria figura
    fig = go.Figure()

    # Paleta de cores
    colors = ['blue', 'green', 'orange', 'purple', 'brown', 'pink', 'gray',
              'cyan', 'magenta', 'lime', 'teal', 'olive', 'navy', 'gold', 'maroon']

    # Adiciona cada feature normalizada
    for i, feature_name in enumerate(features):
        fig.add_trace(go.Scatter(
            x=t,
            y=X_scaled[:, i],
            mode='lines',
            name=feature_name,
            line=dict(color=colors[i % len(colors)], width=2)
        ))

    fig.update_layout(
        title=f"{arquivo} — Features Normalizadas",
        xaxis_title="Tempo [s]",
        yaxis_title="Valor Normalizado (0–1)",
        height=600
        #legend=dict(orientation='v', yanchor='bottom', y=-0.3, xanchor='center', x=0.5)
    )

    fig.show()


#PCA

In [ ]:

for arquivo in arquivos_abre_fecha_emg:
    # Lista de features relevantes
    #selected_features = features_relevantes[arquivo]
    all_features = list(dfs_emg_module[arquivo]["features"].keys())

    # Encontra os índices das features relevantes
    #selected_indices = [all_features.index(f) for f in selected_features]

    # Seleciona colunas de X_scaled correspondentes às features relevantes
    X_scaled_relevante = dfs_emg_module[arquivo]["X_scaled"]

    # Aplica PCA
    pca = PCA(n_components=2)
    X_pca = pca.fit_transform(X_scaled_relevante)

    # Plot
    plt.figure(figsize=(8,5))
    plt.scatter(X_pca[:,0], X_pca[:,1], s=5, alpha=0.7)
    plt.xlabel("PC1")
    plt.ylabel("PC2")
    plt.title(f"PCA 2D - {arquivo} - Todas as Features")
    plt.grid(True)
    plt.show()


# Cluster

## KMeans

In [ ]:
# teste_features = ["amp_en","rms","Band Energy Ratio","Spectral Centroid", "Bandwidth", "f0", "Mag_norm_f0", "bp_50_150", "bp_300_500", "mdf", "mnf", 'wl', "mean_v", "std_v", "ssc", "mavs"]

teste_features = [
    "rms", "Band Energy Ratio", "Spectral Centroid",
    "Mag_norm_f0", "bp_50_150", "mdf", "mnf", "mav",
    "wl", "std_v", "var_v", "ssc", "mavs"
]

teste_features = ["amp_en","rms","Band Energy Ratio","Spectral Centroid", "Bandwidth", "f0",
                 "Mag_norm_f0", "bp_50_150", "bp_300_500", "mdf", "mnf", 'wl',
                "mean_v", "std_v", "ssc", "mavs"]


teste_features = ['rms', 'Spectral Centroid', 'Bandwidth', 'f0', 'Mag_norm_f0','mav', 'wl',
 'mean_v', 'std_v', 'var_v', 'ssc', 'mavs']


teste_features = [ 'rms','mav', 'wl', 'std_v', 'var_v','mavs']

teste_features = ['amp_en', 'rms','Bandwidth', 'f0','bp_50_150','bp_200_300','bp_300_500','mnf',
                 'mav', 'wl','std_v', 'var_v', 'ssc', 'mavs']


In [ ]:
all_features

In [ ]:
for arquivo in arquivos_abre_fecha_emg:
  # Lista de features relevantes

  all_features = list(dfs_emg_module[arquivo]["features_filtered"].keys())

  selected_features = all_features[:]
  #selected_features = teste_features[:]
  #selected_features =  time_features[:]
  #selected_features.remove("zcr")
  #selected_features.remove("mavs")
  #selected_features.remove("mav")

  # Encontra os índices das features relevantes
  selected_indices = [all_features.index(f) for f in selected_features]

  # Seleciona colunas de X_scaled correspondentes às features relevantes
  X_scaled_relevante = dfs_emg_module[arquivo]["X_scaled"][:, selected_indices]




  kmens = KMeans(n_clusters=2,random_state=42).fit(X_scaled_relevante)
  clusters = kmens.labels_
  # cria um filtro para evitar valores falsos, sendo uma variacao bem rapida de 4 5 amostras, ai isso nos previne disso

  clusters = median_clean(signal=clusters,kernel_size=1)
  clusters = run_length_smooth(labels=clusters,min_run_length=7)
  clusters = 1 - clusters

  dfs_emg_module[arquivo]["Clusters_Kmeans"] = clusters



In [ ]:
#for arquivo in arquivos_abre_fecha_emg[len(arquivos_abre_fecha_emg)//2:]:
#  clusters = dfs_emg_module[arquivo]["Clusters_Kmeans"]
#  print(arquivo)
#  print(clusters)

In [ ]:
for arquivo in arquivos_abre_fecha_emg[:]:
    sr = dfs_emg_module[arquivo]["sr"]
    signal = dfs_emg_module[arquivo]["filtered_Nao_Causal"]
    clusters = dfs_emg_module[arquivo]["Clusters_Kmeans"]

    # Número de frames e tamanho de cada frame
    n_frames = len(clusters)
    frame_times = np.arange(n_frames) * HOP_LENGTH / sr  # tempo inicial de cada frame

    # Para plotar clusters no tempo, repetimos o valor do cluster para cada ponto do frame
    cluster_signal = np.repeat(clusters, HOP_LENGTH)
    cluster_time = np.arange(len(cluster_signal)) / sr  # tempo correspondente

    # Ajusta o tamanho para coincidir com o sinal real (em caso de padding final)
    #cluster_signal = cluster_signal[:len(signal)]
    #cluster_time = cluster_time[:len(signal)]

    fig = go.Figure()

    # Plot do sinal EMG
    fig.add_trace(go.Scatter(
        x=np.arange(len(signal))/sr,
        y=signal,
        mode='lines',
        name='EMG',
        line=dict(color='blue', width=1)
    ))

    # Plot do cluster (como linha escalonada)
    fig.add_trace(go.Scatter(
        x=cluster_time,
        y=cluster_signal * np.max(signal),  # escala para coincidir visualmente com o sinal
        mode='lines',
        name='Cluster',
        line=dict(color='red', width=2, dash='dot')
    ))

    fig.update_layout(
        title=f"{arquivo} — Sinal EMG e Cluster por Frame - Kmeans",
        xaxis_title="Tempo [s]",
        yaxis_title="Amplitude",
        height=500
    )

    fig.show()


## Gaussian Mixture Model

In [ ]:
from sklearn.mixture import GaussianMixture

for arquivo in arquivos_abre_fecha_emg:
  # Lista de features relevantes

  all_features = list(dfs_emg_module[arquivo]["features_filtered"].keys())

  selected_features = all_features[:]
  #selected_features = teste_features[:]
  #selected_features =  time_features[:]
  #selected_features.remove("zcr")
  #selected_features.remove("mavs")
  #selected_features.remove("mav")

  # Encontra os índices das features relevantes
  selected_indices = [all_features.index(f) for f in selected_features]

  # Seleciona colunas de X_scaled correspondentes às features relevantes
  X_scaled_relevante = dfs_emg_module[arquivo]["X_scaled"][:, selected_indices]


  n = 2

  gmm = GaussianMixture(n_components=n, random_state=42)
  gmm.fit(X_scaled_relevante)
  clusters = gmm.predict(X_scaled_relevante)

  # cria um filtro para evitar valores falsos

  #clusters = median_clean(signal=clusters,kernel_size=3)
  clusters = run_length_smooth(labels=clusters,min_run_length=15)
  #clusters = median_clean(signal=clusters,kernel_size=3)
  clusters = 1 - clusters

  dfs_emg_module[arquivo]["Clusters_GMM"] = clusters

In [ ]:

for nome in arquivos_abre_fecha_emg:
  if nome == "emg__modulo_abre_fecha_dedao_resto_aberto_10s_3.csv":
    print(f"Alterados na mão: Dataset -> {nome} \n{(len(dfs_emg_module[nome]["Clusters_GMM"][423:440])/len(dfs_emg_module[nome]["Clusters_GMM"][:]))*100} %")
    # faixa que estão os clusters errados [423:440]
    # explicado no final do arquivo
    for i in range(423,440):
      dfs_emg_module[nome]["Clusters_GMM"][i] = 1-dfs_emg_module[nome]["Clusters_GMM"][i]

  if nome == "emg__modulo_abre_fecha_dedao_resto_aberto_10s_4.csv":
    print(f"Alterados na mão: Dataset -> {nome} \n{(len(dfs_emg_module[nome]["Clusters_GMM"][418:431])/len(dfs_emg_module[nome]["Clusters_GMM"][:]))*100} %")
    # faixa que estão os clusters errados [418:431]
    # explicado no final do arquivo
    for i in range(418,431):
      dfs_emg_module[nome]["Clusters_GMM"][i] = 1-dfs_emg_module[nome]["Clusters_GMM"][i]

  if nome == "emg__modulo_abre_fecha_dedao_resto_aberto_10s_5.csv":
    print(f"Alterados na mão: Dataset -> {nome} \n{(len(dfs_emg_module[nome]["Clusters_GMM"][79:82])/len(dfs_emg_module[nome]["Clusters_GMM"][:]))*100} %")
    # faixa que estão os clusters errados [79:82]
    # explicado no final do arquivo
    for i in range(79,82):
      dfs_emg_module[nome]["Clusters_GMM"][i] = 1-dfs_emg_module[nome]["Clusters_GMM"][i]



In [ ]:
for arquivo in arquivos_abre_fecha_emg:
    sr = dfs_emg_module[arquivo]["sr"]
    signal = dfs_emg_module[arquivo]["filtered_Nao_Causal"]
    clusters = dfs_emg_module[arquivo]["Clusters_GMM"]

    # Número de frames e tamanho de cada frame
    n_frames = len(clusters)
    frame_times = np.arange(n_frames) * HOP_LENGTH / sr  # tempo inicial de cada frame

    # Para plotar clusters no tempo, repetimos o valor do cluster para cada ponto do frame
    cluster_signal = np.repeat(clusters, HOP_LENGTH)
    cluster_time = np.arange(len(cluster_signal)) / sr  # tempo correspondente

    # Ajusta o tamanho para coincidir com o sinal real (em caso de padding final)
    cluster_signal = cluster_signal[:len(signal)]
    cluster_time = cluster_time[:len(signal)]

    fig = go.Figure()

    # Plot do sinal EMG
    fig.add_trace(go.Scatter(
        x=np.arange(len(signal))/sr,
        y=signal,
        mode='lines',
        name='EMG',
        line=dict(color='blue', width=1)
    ))

    # Plot do cluster (como linha escalonada)
    fig.add_trace(go.Scatter(
        x=cluster_time,
        y=cluster_signal * np.max(signal),  # escala para coincidir visualmente com o sinal
        mode='lines',
        name='Cluster',
        line=dict(color='red', width=2, dash='dot')
    ))

    fig.update_layout(
        title=f"{arquivo} — Sinal EMG e Cluster por Frame - GMM",
        xaxis_title="Tempo [s]",
        yaxis_title="Amplitude",
        height=500
    )

    fig.show()


## DBSCAN (Density-Based Spatial Clustering of Applications with Noise)

In [ ]:
from sklearn.cluster import DBSCAN

for arquivo in arquivos_abre_fecha_emg:
    #selected_features = features_relevantes[arquivo]
    all_features = list(dfs_emg_module[arquivo]["features_filtered"].keys())
    #selected_indices = [all_features.index(f) for f in selected_features]

    #X_scaled_relevante = dfs_emg_module[arquivo]["X_scaled"][:, selected_indices]
    X_scaled_relevante = dfs_emg_module[arquivo]["X_scaled"][:]

    # Parâmetros iniciais (ajustáveis)
    eps = 1  # distância máxima entre pontos vizinhos
    min_samples = 10  # mínimo de pontos para formar um cluster

    dbscan = DBSCAN(eps=eps, min_samples=min_samples)
    clusters = dbscan.fit_predict(X_scaled_relevante)

    dfs_emg_module[arquivo]["Clusters_DBSCAN"] = clusters

In [ ]:
for arquivo in arquivos_abre_fecha_emg:
    sr = dfs_emg_module[arquivo]["sr"]
    signal = dfs_emg_module[arquivo]["filtered"]
    clusters = dfs_emg_module[arquivo]["Clusters_DBSCAN"]

    # Número de frames e tamanho de cada frame
    n_frames = len(clusters)
    frame_times = np.arange(n_frames) * HOP_LENGTH / sr  # tempo inicial de cada frame

    # Para plotar clusters no tempo, repetimos o valor do cluster para cada ponto do frame
    cluster_signal = np.repeat(clusters, HOP_LENGTH)
    cluster_time = np.arange(len(cluster_signal)) / sr  # tempo correspondente

    # Ajusta o tamanho para coincidir com o sinal real (em caso de padding final)
    cluster_signal = cluster_signal[:len(signal)]
    cluster_time = cluster_time[:len(signal)]

    fig = go.Figure()

    # Plot do sinal EMG
    fig.add_trace(go.Scatter(
        x=np.arange(len(signal))/sr,
        y=signal,
        mode='lines',
        name='EMG',
        line=dict(color='blue', width=1)
    ))

    # Plot do cluster (como linha escalonada)
    fig.add_trace(go.Scatter(
        x=cluster_time,
        y=cluster_signal * np.max(signal),  # escala para coincidir visualmente com o sinal
        mode='lines',
        name='Cluster',
        line=dict(color='red', width=2, dash='dot')
    ))

    fig.update_layout(
        title=f"{arquivo} — Sinal EMG e Cluster por Frame",
        xaxis_title="Tempo [s]",
        yaxis_title="Amplitude",
        height=500
    )

    fig.show()


## Setar Estado

In [ ]:
len("emg__modulo_aberto_")

In [ ]:
len("emg__modulo_fecha_dedao_resto_aberto_")

In [ ]:
arquivo[0:19]

In [ ]:
for arquivo in arquivos_abre_fecha_emg:
    #selected_features = features_relevantes[arquivo]
    all_features = list(dfs_emg_module[arquivo]["features_filtered"].keys())
    #selected_indices = [all_features.index(f) for f in selected_features
    n = len(dfs_emg_module[arquivo]["X_scaled"])
    if arquivo[:19] == "emg__modulo_aberto_":
      clusters = np.zeros(n, dtype=int)
      print(f"Arquivo: {arquivo} setado como {clusters}")
    elif arquivo[:37] == "emg__modulo_fecha_dedao_resto_aberto_":
      clusters = np.ones(n, dtype=int)
      print(f"Arquivo: {arquivo} setado como {clusters}")
    else:
      print(f"Não entrou no IF")
      break

    dfs_emg_module[arquivo]["Label"] = clusters

In [ ]:
dfs_emg_module[arquivo]["Label"]

Aplicar MinMaxScaler apenas para o clustering
Isso garante que todas as features tenham o mesmo peso na hora de formar os clusters.


Obter os rótulos dos clusters com base nesse dataset escalado.


Juntar os rótulos ao dataset original (não escalado)
Assim, você preserva os valores reais das features, o que pode ser útil para interpretação, visualização ou uso posterior.


Aplicar MinMaxScaler no dataset completo (com os rótulos)
Isso pode ser feito se você quiser alimentar um modelo de ML supervisionado depois, por exemplo.

In [ ]:
lista_frames = []

for arquivo in arquivos_abre_fecha_emg:
  signal = dfs_emg_module[arquivo]["filtered"]
  lista_signal = []
  for i in range(0, len(signal) - FRAME_SIZE + 1, HOP_LENGTH):
      wnd = signal[i:i+FRAME_SIZE]

      lista_signal.append(wnd)
  lista_frames.append(lista_signal)




In [ ]:
#rms_calculator(lista_frames[2][-1]*hann_wnd)

In [ ]:
#len(lista_frames[2])

In [ ]:
(len(dfs_emg_module[arquivo]["filtered"]))//HOP_LENGTH

## Features com o Cluster

In [ ]:
## Todos os Arquivos

for arquivo in arquivos_emg:
    sr = dfs_emg_module[arquivo]["sr"]
    #features = features_relevantes[arquivo]
    #features = time_features[:]
    #features.remove("zcr")
    #features.remove("mavs")
    #features.remove("mav")
    features = list(dfs_emg_module[arquivo]["features_filtered"].keys())
    # Obtém matriz normalizada e tamanho
    X_scaled = dfs_emg_module[arquivo]["X_scaled"]
    n_frames = X_scaled.shape[0]
    t = np.arange(n_frames) * HOP_LENGTH / sr

    # Cria figura
    fig = go.Figure()

    # Paleta de cores
    colors = ['blue', 'green', 'orange', 'purple', 'brown', 'pink', 'gray',
              'cyan', 'magenta', 'lime', 'teal', 'olive', 'navy', 'gold', 'maroon']

    # Adiciona cada feature normalizada
    for i, feature_name in enumerate(features):
        fig.add_trace(go.Scatter(
            x=t,
            y=X_scaled[:, i],
            mode='lines',
            name=feature_name,
            line=dict(color=colors[i % len(colors)], width=2)
        ))
    fig.add_trace(go.Scatter(
        x=t,
        y=dfs_emg_module[arquivo]["Clusters_Kmeans"],
        mode='lines',
        name="Kmeans",
        line=dict(color="red", width=2)
    ))

    fig.add_trace(go.Scatter(
        x=t,
        y=dfs_emg_module[arquivo]["Clusters_GMM"],
        mode='lines',
        name="GMM",
        line=dict(color="blue", width=2)
    ))
    fig.update_layout(
        title=f"{arquivo} — Features Normalizadas",
        xaxis_title="Tempo [s]",
        yaxis_title="Valor Normalizado (0–1)",
        height=600
        #legend=dict(orientation='v', yanchor='bottom', y=-0.3, xanchor='center', x=0.5)
    )

    fig.show()


amp_en,rms,Band Energy Ratio,Spectral Centroid,  Mag_norm_f0, bp_50_150, mdf, mnf, wav,wl, std_v,var_v, ssc, mavs

# Salvar

## Abre e Fecha

In [ ]:
arquivos_abre_fecha_emg

In [ ]:
 ## Salvar os arquivos abre e fecha

from sklearn.metrics import adjusted_rand_score

for arquivo in arquivos_abre_fecha_emg:
  print(arquivo)
  arq_selecionado = input("Nome do Arquivo: \n")

  if arq_selecionado in arquivo:
    features_headers = list(dfs_emg_module[arq_selecionado]["features_filtered"].keys())
    features_headers.extend(["Label"])

    array_abre_fecha_2 = dfs_emg_module[arq_selecionado]["X_Brutas"]
    cluster_abre_fecha_2_label = dfs_emg_module[arq_selecionado]["Label"]

    # converter cluster em coluna (336, 1)
    cluster_abre_fecha_2_label = np.array(cluster_abre_fecha_2_label).reshape(-1, 1)

    # concatenar lado a lado (axis=1)
    concatenado_abre_fecha_2 = np.concatenate((array_abre_fecha_2, cluster_abre_fecha_2_label), axis=1)



    print(concatenado_abre_fecha_2.shape)

    df_features = pd.DataFrame(concatenado_abre_fecha_2,columns=features_headers)

    salvar = input("Salvar? (S/N)")
    if salvar == "S":
      # Salva em CSV
      df_features.to_csv(path+"Features Datasets/"+arq_selecionado+"Features.csv", index=False)
      print("Arquivo path+'Features Datasets/'"+arq_selecionado+"_Features.csv" +" salvo com sucesso!")

  else:
    print("Não Criou o DataFrame")



In [ ]:
df_features

# Processo de Cluster, Individual:

Neste Contexto o importante é entender como otimizar o processo de cluster de cada dataset, neste não podemos alterar os filtros na frequência.
É então interessante analisar como podemos ter o melhor resultado dos labels individualmente para cada dataset, e salvar o mesmo como csv para usarmos no modelo supervisionado posteriormente.
Este ajuste passa por:

- **Filtros nas Features:** Entender os parâmetros ótimos para os filtros das Features (mediana e média móvel), além de analisar se é interessante ou não ter o elevado ao quadrado.

- **Filtros no Cluster:** Podemos rodar um filtro no próprio cluster, para evitar falsos positivos ou negativos, para este caso usamos o mediana e run_length_smooth, quando rodamos este no cluster podemos sim escolhermos se usamos ou não, ou alterar parâmetros do mesmo para cada dataset

- **Escolha das Features:** Pode Acontecer que para Datasets diferentes terem resultados diferentes as combinações de features utilizadas tem que ser diferentes, importante aqui é usar todas as features quando formos criar o csv, pois será utilizado no modelo supervisionado.

- **Alteração Manual**: Como último caso podemos alterar alguns labels de forma manual, se e somente se, este forem casos isolados e tentamos todas as alternativas acima não obtendo sucesso.

Sendo assim, devemos ao encontrar a configuração perfeita para cada dataset, salva-lo imediatamente, e documentar os parâmetros usados para deixar o resultado ótimo. Como iremos juntar depois o dataset das features cru, não tem problema utilizar estes parâmetros diferentes, depois no modelo supervisionado analisamos.

Devemos então salvar as features **BRUTAS**, sem processamento.

# Parametros Ótimos por Dataset:

Nesse apenas setamos como 1 o dataset de contração e 0 o dataset de repouso

Chamando a coluna de Label

# 🧠 Etapas de Processamento e Análise de Dados EMG

O conjunto de dados EMG foi processado em múltiplas etapas para extração, filtragem e preparação das *features* relevantes.  
Essas etapas garantem que os dados estejam consistentes, livres de ruídos e adequados para análise estatística e modelagem de aprendizado de máquina.

---

## ⚙️ 1. Filtragem e Normalização

Cada arquivo EMG passou por um processo de pré-tratamento das *features*, que envolveu:

- **Suavização com média móvel** (`simple_moving_average`): reduz flutuações rápidas e melhora a estabilidade do sinal.
- **Filtro de mediana** (`median_clean`): remove picos abruptos de ruído preservando as bordas das contrações musculares.
- **Operação quadrática** (`value = value ** 2`): realça amplitudes maiores, destacando regiões de maior atividade elétrica.

Após a filtragem, todas as *features* foram truncadas para o mesmo comprimento temporal e empilhadas em uma matriz com o formato:
\[
X = [f_1, f_2, f_3, ..., f_n]
\]
onde cada coluna representa uma *feature* e cada linha um *frame* temporal do sinal.

Em seguida, aplicou-se a **normalização Min-Max**:
\[
x_{norm} = \frac{x - x_{min}}{x_{max} - x_{min}}
\]
de forma a mapear todos os valores para o intervalo [0, 1], garantindo escala uniforme entre atributos e evitando que *features* com maiores magnitudes dominem o agrupamento.

---

## 🧩 2. Clusterização Não Supervisionada

Com as *features* filtradas e normalizadas, aplicaram-se dois métodos distintos de aprendizado não supervisionado para identificar padrões de ativação muscular:

### 🟢 K-Means
- Número de clusters: **2**
- Agrupa os dados com base na distância euclidiana.
- Útil para distinguir entre estados, como “músculo contraído” e “músculo relaxado”.

### 🔵 Gaussian Mixture Model (GMM)
- Número de componentes: **2**
- Modelo probabilístico que permite clusters com formatos não necessariamente esféricos.
- Usa o método `predict()` para estimar a probabilidade de cada amostra pertencer a um cluster.

Os rótulos de cluster obtidos foram adicionados às matrizes de *features* sob as colunas:
- `Clusters_Kmeans`
- `Clusters_GMM`

---

## 🧮 3. Construção do Dataset Final

Após a clusterização, os conjuntos correspondentes a diferentes gravações EMG foram unificados.  
O dataset resultante foi estruturado da seguinte forma:

\[
[X_{features} \;|\; Cluster_{KMeans} \;|\; Cluster_{GMM}]
\]

Esse formato facilita a utilização posterior em tarefas supervisionadas de classificação.

---

## 💾 4. Exportação do Dataset

O dataset final, contendo todas as *features* filtradas e normalizadas junto aos rótulos de clusterização, foi exportado para o arquivo:

